Idea for tuning:

need to specify which metrics need to be computed, which one needs to be minimized, and how large k is

* build k splits into train and test data
* for each train-test-split:
    * build a parameter grid using cls.tuning_params. (For each class we have to find out what a good grid would be.)
    * for each combination in the grid: 
        * translate api-response into f,m,u and store in the column gender_infered (GOT THIS FAR!!!)
        * compute all provided metrics on the resulting test_data DF restricted to training data
        * store relation grid-point:metrics
    * select grid_point which minimizes specified metric and compute all provided metrics on test set
    * store the metrics on test set
* compute average of metrics on all test sets for all metrics

In [1]:
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from evaluators import *
from evaluator import *
from itertools import product
from collections import OrderedDict
import numpy as np

In [2]:
data_source = 'all'
service_name = GenderizeIoEvaluator
evaluator = service_name(data_source)

In [3]:
evaluator.load_data(evaluated=True)

In [4]:
evaluator.test_data.head()

,first_name,middle_name,last_name,full_name,gender,origin,api_count,api_gender,api_name,api_probability,gender_infered
0,pierre,paul,grivel,pierre paul grivel,m,zbmath,5.0,male,pierre-paul,1.0,m
1,raul,,serapioni,raul serapioni,m,zbmath,821.0,male,raul,1.0,m
2,adriano,,moura,adriano moura,m,zbmath,166.0,male,adriano,0.99,m
3,ralf,,kieser,ralf kieser,m,zbmath,86.0,male,ralf,1.0,m
4,teppei,,ariyoshi,teppei ariyoshi,u,zbmath,,,teppei,,u


In [5]:
# we have to ignore 'u' in column 'gender'
evaluator.remove_rows_with_unknown_gender()

In [6]:
evaluator.tuning_params

('api_count', 'api_probability')

In [7]:
evaluator.build_parameter_grid([1,10, 50, 100], [0.5, 0.7, 0.8, 0.9])

[{'api_count': 1, 'api_probability': 0.5},
 {'api_count': 1, 'api_probability': 0.7},
 {'api_count': 1, 'api_probability': 0.8},
 {'api_count': 1, 'api_probability': 0.9},
 {'api_count': 10, 'api_probability': 0.5},
 {'api_count': 10, 'api_probability': 0.7},
 {'api_count': 10, 'api_probability': 0.8},
 {'api_count': 10, 'api_probability': 0.9},
 {'api_count': 50, 'api_probability': 0.5},
 {'api_count': 50, 'api_probability': 0.7},
 {'api_count': 50, 'api_probability': 0.8},
 {'api_count': 50, 'api_probability': 0.9},
 {'api_count': 100, 'api_probability': 0.5},
 {'api_count': 100, 'api_probability': 0.7},
 {'api_count': 100, 'api_probability': 0.8},
 {'api_count': 100, 'api_probability': 0.9}]

In [8]:
grid = evaluator.build_parameter_grid([1, 10, 100], [0.7, 0.8, 0.9])

In [9]:
# TODO: integrate into code for this specific evaluator
for col in evaluator.tuning_params:
    evaluator.test_data[col] = evaluator.test_data[col].replace({'':None}) # None is coded as empty string
    evaluator.test_data[col] = evaluator.test_data[col].astype(float)

In [12]:
train_test_splits = evaluator.build_train_test_splits(evaluator.test_data, 3)

In [13]:
train_test_splits

[(array([   0,    1,    2, ..., 5787, 5788, 5789]),
  array([   3,    4,    5, ..., 5779, 5784, 5785])),
 (array([   0,    2,    3, ..., 5785, 5786, 5787]),
  array([   1,    6,    7, ..., 5780, 5788, 5789])),
 (array([   1,    3,    4, ..., 5785, 5788, 5789]),
  array([   0,    2,    8, ..., 5783, 5786, 5787]))]

In [54]:
# for testing
len(train_test_splits)

3

In [55]:
# for testing
len(train_test_splits[0])

2

In [56]:
# for testing
for train_index, test_index in train_test_splits:
    print(train_index, test_index)

[   0    1    2 ..., 5787 5788 5789] [   3    4    5 ..., 5779 5784 5785]
[   0    2    3 ..., 5785 5786 5787] [   1    6    7 ..., 5780 5788 5789]
[   1    3    4 ..., 5785 5788 5789] [   0    2    8 ..., 5783 5786 5787]


In [68]:
# test sorting of param_to_error_mapping
d = {(0.5, 10): (0.03, 0.02), (0.7, 100): (0.01, 0.05), (0.7, 1): (0.006, 0.01)}
sorted(d.items(), key=lambda x: x[1][1]) # sort by second item in value

[((0.7, 1), (0.006, 0.01)),
 ((0.5, 10), (0.03, 0.02)),
 ((0.7, 100), (0.01, 0.05))]

In [80]:
# compute predicted error
param_1, param_2 = evaluator.tuning_params[0], evaluator.tuning_params[1]
kfold_errors = [] # errors on each of the k test sets for the optimal function on corresponding train set

for train_index, test_index in train_test_splits:
    print(train_index, test_index)
    param_to_error_mapping = {}
    for item in grid: 
        evaluator._translate_api_response(**item)
        conf_matrix_train = evaluator.compute_confusion_matrix(evaluator.test_data.loc[train_index,:])
        conf_matrix_test = evaluator.compute_confusion_matrix(evaluator.test_data.loc[test_index,:])
        error_train = evaluator.build_error_without_unknown(conf_matrix_train)
        error_test = evaluator.build_error_without_unknown(conf_matrix_test)
        param_to_error_mapping[(item[param_1], item[param_2])] = (error_train, error_test)
        # print(param_to_error_mapping)
    d = OrderedDict(param_to_error_mapping.items(), key=lambda x:x[1][1])
    items = list(d.items())
    print("minimal train error:", items[0][1][0], "corresponding test error:", items[0][1][1])
    test_error = items[0][1][1]
    kfold_errors.append(test_error)

[   0    1    2 ..., 5787 5788 5789] [   3    4    5 ..., 5779 5784 5785]
minimal train error: 0.0153846153846 corresponding test error: 0.0211800302572
[   0    2    3 ..., 5785 5786 5787] [   1    6    7 ..., 5780 5788 5789]
minimal train error: 0.0180995475113 corresponding test error: 0.0157303370787
[   1    3    4 ..., 5785 5788 5789] [   0    2    8 ..., 5783 5786 5787]
minimal train error: 0.0184418517125 corresponding test error: 0.015037593985


In [81]:
kfold_errors

[0.02118003025718608, 0.015730337078651686, 0.015037593984962405]

In [85]:
np.mean(kfold_errors)

0.017315987106933391

## TODO: integrate code below

In [ ]:
def build_train_test_splits(df, cols, n_splits, stratified=False, shuffle=True):
    X = df[cols]
    y = df['gender'] # required only for stratified version
    train_test_splits = []
    if stratified is False:
        kf = KFold(n_splits=n_splits, random_state=1, shuffle=shuffle)
    else:
        kf = StratifiedKFold(n_splits=n_splits, random_state=1, shuffle=shuffle)
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
        y_train, y_test = y[train_index], y[test_index]
        train_test_splits.append((X_train, X_test, y_train, y_test))
    return train_test_splits

In [67]:
X = evaluator.test_data[['full_name', 'api_count', 'api_probability', 'api_gender']].head(50)
y = evaluator.test_data['gender'][:50]

In [75]:
kf = KFold(n_splits=3, random_state=1, shuffle=True)
skf = StratifiedKFold(n_splits=3, random_state=1, shuffle=True)

In [76]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 0  1  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 22 23 24 25 26 28 30
 33 34 37 41 43 44 45 47] TEST: [ 2  3 19 21 27 29 31 32 35 36 38 39 40 42 46 48 49]
TRAIN: [ 0  1  2  3  5  6  7  8  9 11 12 15 16 19 20 21 25 27 29 31 32 35 36 37 38
 39 40 42 43 46 47 48 49] TEST: [ 4 10 13 14 17 18 22 23 24 26 28 30 33 34 41 44 45]
TRAIN: [ 2  3  4 10 13 14 17 18 19 21 22 23 24 26 27 28 29 30 31 32 33 34 35 36 38
 39 40 41 42 44 45 46 48 49] TEST: [ 0  1  5  6  7  8  9 11 12 15 16 20 25 37 43 47]


In [ ]:
X_train.head()

In [78]:
for train_index, test_index in skf.split(X,y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 0  1  4  5  7  8  9 10 11 12 13 14 15 17 18 19 21 23 27 28 29 30 31 32 33
 37 41 42 43 44 45 47 48] TEST: [ 2  3  6 16 20 22 24 25 26 34 35 36 38 39 40 46 49]
TRAIN: [ 0  1  2  3  6  7  9 10 11 13 14 15 16 18 19 20 22 24 25 26 31 32 34 35 36
 38 39 40 42 46 47 48 49] TEST: [ 4  5  8 12 17 21 23 27 28 29 30 33 37 41 43 44 45]
TRAIN: [ 2  3  4  5  6  8 12 16 17 20 21 22 23 24 25 26 27 28 29 30 33 34 35 36 37
 38 39 40 41 43 44 45 46 49] TEST: [ 0  1  7  9 10 11 13 14 15 18 19 31 32 42 47 48]
